In [32]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


In [33]:
path_csv_file = "C:/Users/murat/Desktop/Nlp_delivery2/data.csv"

In [34]:
df = pd.read_csv(path_csv_file)
df.head()

,File Name,Label,Text
0,1.json,undefined,"Kasten basit yaralama suçundan sanık ...'nin,..."
1,10.json,undefined,Taraflar arasında görülen ve yukarıda açı...
2,100.json,undefined,"(5237 S. K. m. 53) (5271 S. K. m. 288, 289, 2..."
3,1000.json,undefined,BÖLGE ADLİYE MAHKEMESİ:... 5. Hukuk Dairesi ...
4,10000.json,undefined,Taraflar arasında görülen davada; Davacı...


In [35]:
label_map = {}
count = 0
for label in df.Label.unique():
    label_map[label] = count
    count += 1
df['NUM_LABEL'] = df.Label.map(label_map)
#print(label_map)
df.head()

,File Name,Label,Text,NUM_LABEL
0,1.json,undefined,"Kasten basit yaralama suçundan sanık ...'nin,...",0
1,10.json,undefined,Taraflar arasında görülen ve yukarıda açı...,0
2,100.json,undefined,"(5237 S. K. m. 53) (5271 S. K. m. 288, 289, 2...",0
3,1000.json,undefined,BÖLGE ADLİYE MAHKEMESİ:... 5. Hukuk Dairesi ...,0
4,10000.json,undefined,Taraflar arasında görülen davada; Davacı...,0


In [36]:
df.Label.value_counts()

undefined                                                                                                                                                                                 12951
kasten yaralama                                                                                                                                                                            1485
uyuşturucu madde ticareti yapma                                                                                                                                                             693
hırsızlık                                                                                                                                                                                   682
tehdit                                                                                                                                                                                      612
                                        

In [37]:
texts = df.Text
label_nums = df.NUM_LABEL

In [38]:
X_train, X_test, y_train, y_test = train_test_split(texts, label_nums, test_size=0.2)

In [39]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(22280,)
(5571,)
(22280,)
(5571,)


In [40]:
X_train

19814            Yukarıda tarih ve numarası yazılı Bölg...
17495            Yukarıda tarih ve numarası yazılı mahk...
3127      (5271 S. K. m. 142) (YİBK 24.05.2019 T. 2017/...
9053      (5271 S. K. m. 48)  Dava ve Karar: 1- Sanıkla...
12708        Taraflar arasındaki alacak davasının mahke...
                               ...                        
9541          Taraflar arasında görülen dava sonucunda ...
20743              Yukarıda tarih ve numarası yazılı Bö...
10985       HÜKÜM : Mahkumiyet      Mahalli mahkemece v...
17994     (5237 S. K. m. 53) (ANY. MAH. 08.10.2015 T. 2...
24392       HÜKÜM : Mahkumiyet    Dosya incelenerek ger...
Name: Text, Length: 22280, dtype: object

In [41]:
def clean_text(text):
    alphabetic_only = [word for word in text.split() if word.isalpha()]
    lower_case_only = [word.lower() for word in alphabetic_only]
    stopwords_tr = set(stopwords.words("turkish"))
    return [word for word in lower_case_only if word not in stopwords_tr]

In [42]:
"""
vect = CountVectorizer(analyzer='word', tokenizer=clean_text)
X_train_dtm = vect.fit_transform(X_train.astype("U"))
X_train_dtm
"""

'\nvect = CountVectorizer(analyzer=\'word\', tokenizer=clean_text)\nX_train_dtm = vect.fit_transform(X_train.astype("U"))\nX_train_dtm\n'

In [43]:
"""
X_test_dtm = vect.transform(X_test.astype("U"))
X_test_dtm
"""

'\nX_test_dtm = vect.transform(X_test.astype("U"))\nX_test_dtm\n'

In [44]:
vect2 = TfidfVectorizer()
vect2.fit(texts.astype("U"))

Train_X_Tfidf = vect2.transform(X_train.astype("U"))
Test_X_Tfidf = vect2.transform(X_test.astype("U"))

In [45]:
model = MultinomialNB()
#model = LogisticRegression(max_iter=1000)
#model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
model.fit(Train_X_Tfidf, y_train)

MultinomialNB()

In [46]:
y_pred_class = model.predict(Test_X_Tfidf)
metrics.accuracy_score(y_pred_class, y_test)

0.48662717644947046

In [47]:
def predict_label(text):
    test = vect.transform(text)
    prediction = model.predict(test)
    return prediction